In [1]:
import random
import os
import pandas as pd
import numpy as np
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
from albumentations.pytorch.transforms import ToTensorV2
from torch.cuda.amp import GradScaler
import cv2
from sklearn.metrics import f1_score

SEED=7777
BATCH_SIZE=16
IMG_SIZE=224
LR_INIT=1e-4
EPOCHS=50

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
device = torch.device('cuda')

In [2]:
all_img_list = glob('./train/*/*')
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[2])
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=SEED)
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [3]:
def hangulFilePathImageRead ( filePath ) :
    
    stream = open( filePath.encode("utf-8") , "rb")
    bytes = bytearray(stream.read())
    numpyArray = np.asarray(bytes, dtype=np.uint8)

    return cv2.imdecode(numpyArray , cv2.IMREAD_UNCHANGED)

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = hangulFilePathImageRead(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [4]:
train_transform = A.Compose([
                            A.Resize(256, 256),
                            A.RandomCrop(224, 224),
                            A.RandomRotate90(),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(256, 256),
                            A.RandomCrop(224, 224),
                            A.RandomRotate90(),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [5]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=False)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, transforms=test_transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
model = timm.create_model('swin_base_patch4_window7_224.ms_in22k_ft_in1k', pretrained=True)
model.head.fc = nn.Linear(1024, 19)
model.to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=LR_INIT, weight_decay=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS-10, eta_min=0, last_epoch=-1)
scaler = torch.cuda.amp.GradScaler()

c:\Users\korea\anaconda3\envs\firesimulation\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
def train(model, optimizer, train_loader, val_loader, scheduler, scaler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, EPOCHS +1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(imgs)
                loss = criterion(output, labels)
    
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if epoch >= 10:
            scheduler.step()
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

@torch.no_grad()
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []


    for imgs, labels in tqdm(iter(val_loader)):
        imgs = imgs.float().to(device)
        labels = labels.type(torch.LongTensor).to(device)
        
        pred = model(imgs)
        
        loss = criterion(pred, labels)
        
        preds += pred.argmax(1).detach().cpu().numpy().tolist()
        true_labels += labels.detach().cpu().numpy().tolist()
        
        val_loss.append(loss.item())
        
    _val_loss = np.mean(val_loss)
    _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [8]:
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, scaler, device)

100%|██████████| 65/65 [00:09<00:00,  6.79it/s]


Epoch [1], Train Loss : [1.14759] Val Loss : [0.85769] Val Weighted F1 Score : [0.72843]


100%|██████████| 65/65 [00:09<00:00,  6.66it/s]


Epoch [2], Train Loss : [0.64097] Val Loss : [0.67044] Val Weighted F1 Score : [0.79102]


100%|██████████| 65/65 [00:09<00:00,  6.63it/s]


...

Epoch [48], Train Loss : [0.00054] Val Loss : [0.76466] Val Weighted F1 Score : [0.86164]


100%|██████████| 65/65 [00:10<00:00,  6.39it/s]


Epoch [49], Train Loss : [0.00136] Val Loss : [0.80881] Val Weighted F1 Score : [0.86407]


100%|██████████| 65/65 [00:10<00:00,  6.30it/s]

Epoch [50], Train Loss : [0.00061] Val Loss : [0.76597] Val Weighted F1 Score : [0.86185]


In [11]:
test = pd.read_csv('./test.csv')
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

@torch.no_grad()
def inference(model, test_loader, device):
    model.eval()
    preds = []
    for imgs in tqdm(iter(test_loader)):
        imgs = imgs.float().to(device)
        
        pred = model(imgs)
        
        preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

preds = inference(infer_model, test_loader, device)

100%|██████████| 50/50 [00:23<00:00,  2.09it/s]


In [15]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.to_csv('./swintransformer_submit.csv', index=False)

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[2])